# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.233950626348                   -0.50    8.0         
  2   -7.250177442628       -1.79       -1.40    1.0   9.09ms
  3   -7.251112554012       -3.03       -1.99    2.0   10.9ms
  4   -7.251147589283       -4.46       -1.99    2.0   12.1ms
  5   -7.251321639814       -3.76       -2.53    1.0   10.3ms
  6   -7.251337497968       -4.80       -3.05    1.0   9.09ms
  7   -7.251338665363       -5.93       -3.54    2.0   10.9ms
  8   -7.251338790470       -6.90       -4.03    2.0   11.3ms
  9   -7.251338797520       -8.15       -4.65    2.0   10.9ms
 10   -7.251338798577       -8.98       -4.88    3.0   13.5ms
 11   -7.251338798683       -9.98       -5.45    1.0   10.2ms
 12   -7.251338798696      -10.88       -5.52    3.0   12.8ms
 13   -7.251338798703      -11.13       -5.97    1.0   11.0ms
 14   -7.251338798704      -12.00       -6.66    2.0   12.2ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03889410490751463
[ Info: Arnoldi iteration step 2: normres = 0.7290199400180605
[ Info: Arnoldi iteration step 3: normres = 0.7246634052110912
[ Info: Arnoldi iteration step 4: normres = 0.4465662475932176
[ Info: Arnoldi iteration step 5: normres = 0.3063365174035531
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.72e-02, 7.47e-02, 1.71e-01, 2.37e-01, 3.70e-02)
[ Info: Arnoldi iteration step 6: normres = 0.40987049112227447
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.29e-02, 1.26e-01, 3.74e-01, 7.17e-02, 3.04e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09042554472590801
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (6.10e-04, 1.82e-02, 2.61e-02, 2.63e-02, 7.55e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10598280784257909
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.80e-